In [1]:
%load_ext autoreload
%autoreload 2
from glob import glob
import os
from pathlib import Path
import re
import sys
import yaml
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
from func.utils import get_categorical_features, read_pkl_gzip, to_pkl_gzip, parallel_load_data, get_filename
from func.ml_utils import Classifier

In [12]:
COLUMN_ID = 'TransactionID'
COLUMN_DT = 'TransactionDT'
COLUMN_TARGET = 'isFraud'
COLUMNS_IGNORE = [COLUMN_ID, COLUMN_DT, COLUMN_TARGET]

train_feature_paths = glob('../feature/raw_main/*_train.gz')
feature_names = [re.search(r'/([^/.]*)_train.gz', path).group(1) for path in train_feature_paths]

train_base = read_pkl_gzip('../input/base_train.gz')
Y = train_base[COLUMN_TARGET]
train_base.drop(COLUMN_TARGET, axis=1, inplace=True)

In [9]:
feature = 'C1'
train_feature_path = f'../feature/raw_main/{feature}_train.gz'
c1_train_feature = read_pkl_gzip(train_feature_path)

In [14]:
length = int(len(train_feature)*0.8)
params = {}
metric = 'auc'
model_type = 'lgb'

for c1_value in range(2,6,1):

    score_list = []
    for feature in tqdm(feature_names):
        if feature in COLUMNS_IGNORE:
            continue
            
        train_feature_path = f'../feature/raw_main/{feature}_train.gz'
        train_feature = read_pkl_gzip(train_feature_path)
        
        x_train = pd.DataFrame(train_feature[:length], columns=[feature])
        y_train = Y[:length]
        x_valid = pd.DataFrame(train_feature[length:], columns=[feature])
        y_valid = Y[length:]
        
        x_train['C1'] = c1_train_feature[:length]
        x_valid['C1'] = c1_train_feature[length:]
        x_train = x_train[x_train['C1']>=c1_value]
        x_valid = x_valid[x_valid['C1']>=c1_value]
        y_train = y_train.iloc[x_train.index]
        y_valid = y_valid.iloc[x_valid.index]
    
        try:
            score, oof_pred, test_pred, feim, _ = Classifier(
                model_type=model_type,
                x_train=x_train,
                y_train=y_train,
                x_valid=x_valid,
                y_valid=y_valid,
                x_test=[],
                params=params,
            )
        except ValueError:
            continue
        score_list.append([feature, score])
        
    score_result = pd.DataFrame(score_list, columns=['feature', 'score'])
    score_result.sort_values(by='score', ascending=False, inplace=True)
    
    score_result.to_csv(f'../eda/103__eda_single_feature_train_AUC_holdout8-2__C1-{c1_value}over.csv', index=False)

  0%|          | 0/285 [00:00<?, ?it/s]/home/yryrgogo/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1225: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is []
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/home/yryrgogo/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1225: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is []
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Training until validation scores don't improve for 50 rounds.


  0%|          | 1/285 [00:02<13:58,  2.95s/it]

Early stopping, best iteration is:
[100]	valid_0's auc: 0.792793


/home/yryrgogo/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1225: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is []
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/home/yryrgogo/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1225: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is []
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


...

Early stopping, best iteration is:
[71]	valid_0's auc: 0.846034
Training until validation scores don't improve for 50 rounds.


/home/yryrgogo/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1225: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is []
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/home/yryrgogo/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1225: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is []
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
 99%|█████████▉| 283/285 [01:10<00:00,  2.80it/s]

Early stopping, best iteration is:
[12]	valid_0's auc: 0.83692
Training until validation scores don't improve for 50 rounds.


/home/yryrgogo/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1225: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is []
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/home/yryrgogo/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1225: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is []
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
100%|█████████▉| 284/285 [01:10<00:00,  2.55it/s]/home/yryrgogo/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1225: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['ProductCD']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/home/yryrgogo/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1225: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['ProductCD']
  'New categorical_feature is {}'.

Early stopping, best iteration is:
[48]	valid_0's auc: 0.859542


In [70]:
score_result['score_round'] = score_result['score'].round(2)
score_cnt = score_result['score_round'].value_counts()
score_cnt

0.68    61
0.69    47
0.53    35
0.67    34
0.70    26
0.52    25
0.51    18
0.54    18
0.50    14
0.65    13
0.58    12
0.64    11
0.57    10
0.66     9
0.59     9
0.62     8
0.63     7
0.56     7
0.61     5
0.55     3
0.60     3
0.72     1
0.71     1
Name: score_round, dtype: int64

In [74]:
score_result.to_csv('../eda/103__eda_single_feature_train_AUC_holdout8-2.csv', index=False)

In [87]:
from kaggle_utils import move_feature
raw_score_features = score_result[score_result['score']<0.55]['feature'].values
move_feature(raw_score_features, 'raw_main', 'trush')

/home/yryrgogo/github/ieee-fraud/feature/raw_main/V68_train.gz


SystemExit: 

/home/yryrgogo/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3333: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [5]:
train_paths = glob('../feature/raw_main/*_train.gz')
test_paths = glob('../feature/raw_main/*_test.gz')

train_paths = [path for path in train_paths if path.count('DT') or path.count('day_no')]
test_paths  = [path for path in test_paths  if path.count('DT') or path.count('day_no')]

train_df = parallel_load_data(train_paths)
test_df = parallel_load_data(test_paths)
print(train_df.shape)
print(test_df.shape)

train_df['is_train'] = True
test_df['is_train']  = False
result = pd.concat([train_df, test_df], axis=0)

DIR_INPUT = Path('../input')
file_name = 'ieee__result_single_feature_AUC_055-Over.csv'
file_path = DIR_INPUT / file_name

# result.to_csv(file_path, index=False)

(590540, 2)
(506691, 2)


In [106]:
from gcp_utils import gcs_upload

gcs_upload(blob_name=file_name, file_path=file_path)